In [8]:
# Define Helper Functions

#Function to find a list of terms in a df
def find_vars(df, list_of_terms):
    regex_version = '('+('|').join(list_of_terms)+')'
    var_filter = df.index.str.contains(regex_version, flags=re.IGNORECASE)
    vars_list = list(df[var_filter].index)
    return vars_list

#Function to find only Categorical values with Yes or No values
def filter_y_n(df):
    df_func = df[df['categorical'] == True]
    df_y_n = df_func[df_func['categoryValues'].apply(lambda x: 'Yes' in x or 'YES' in x or 'y' in x or 'Y' in x)]
    return df_y_n

# Function to find only Continous Variables, can add min and max values here too
def filter_continous(df):
    df_func = df[df['categorical'] == False]
    return df_func

#Create function to clean the messy column names

def simplify_vars_df(df):
    cleaned_vars = []
    for vars in df.columns:
        if vars[-0] == '\\':
            
            # this string manipulation *might* be different across 
            # studies but has been consistent in all the studies I have seen so far
            vars = vars[:-1]
            clean_var = vars[vars.rindex('\\'):]
            clean_var = clean_var[1:]
            cleaned_vars.append(clean_var)
        else:
            cleaned_vars.append(vars)
            pass
        
        intermediate_list = []
        for i in range(len(cleaned_vars)):
            if cleaned_vars[i] in intermediate_list:
                cleaned_vars[i] = cleaned_vars[i] + ' Duplicate ' + str(i + 1)
                
            intermediate_list.append(cleaned_vars[i])
    return cleaned_vars


# Will use this function do filter variables easily before querying
def simplify_vars_list(df):
    cleaned_vars_dict = {}
    for i in range(len(df)):
        vars = df[i]
        if vars[-0] == '\\':
            
            # this string manipulation *might* be different across 
            # studies but has been consistent in all the studies I have seen so far
            
            # Basically in these studies everything except the txt after the last slash is the same
            # so use that to simplify
            vars = vars[:-1]
            clean_var = vars[vars.rindex('\\'):]
            clean_var = clean_var[1:]
            cleaned_vars_dict[i] = clean_var
        else:
            cleaned_vars_dict[i] = vars
            pass
    return cleaned_vars_dict

# find a better way to do this, maybe use Regex

#Function to return the keys for the simplified values

def findKeys(dict1 ,str_list):
    # make a flag
    import pandas as pd
    regex_version = '('+('|').join(str_list)+')'
    keys = []
    TF_List = pd.Index(dict1.values()).str.contains(regex_version, case = False)
    for i in range(len(TF_List)):
        if TF_List[i] == True:
            keys.append(list(dict1.keys())[i])
    return keys

In [3]:
def pull_study(study_name, resource):
    
    
    pull_result = resource.dictionary().find(study_name).DataFrame()
    
    return pull_result

def original_var_str(simplified_vars, original_df, str_interest):
    # Take a data frame of simplified variable names, and string of interest and find the 
    # corresponding original var names
    
    #add complexity here
    original_vars_of_interest = []
    simplified_vars_of_interest = findKeys(simplified_vars, str_interest)
    for i in simplified_vars_of_interest:
        original_vars_of_interest.append(original_df.index[i])
        
    return original_vars_of_interest

def start_query(var_list, resource):
    # start query. with a list of variable names
    query = resource.query()
    query.anyof().add(var_list)
    query_results = query.getResultsDataFrame(low_memory = False)
    query_vars_cleaned = simplify_vars_df(query_results)
    query_results.columns = query_vars_cleaned
    return query_results

def choose_data_type(results_df, category):
    # filter data frame for either categorical or continous data, can add complexity here
    if category == 'object':
        filtered_df = results_df.select_dtypes(include = ['object']).columns
        return filtered_df
        
    elif category == 'number':
        filtered_df = results_df.select_dtypes(include = ['number']).columns
        return filtered_df
        
    else:
        print('Please enter a valid data category and retry')
        

In [12]:
def pull_var_from_study(study_name, str_interest, resource):
    # Find a way to make str_interest a list of strings
    import pandas as pd
    study_pull = pull_study(study_name, resource)
    study_simplified_vars = simplify_vars_list(study_pull.index)
    study_vars_interest = original_var_str(study_simplified_vars, study_pull, str_interest)
    study_q_results = start_query(study_vars_interest, resource)
    var_type = input('Do you want Categorical(c) or Numerical Data(n)?')
    if var_type == 'c':
        study_filtered = choose_data_type(study_q_results, 'object')
        print('')
        for i in range(len(study_filtered)):
            column_i = study_q_results[study_filtered[i]].unique()
            if len(column_i) > 5:
                values = column_i[0:4]
            else:
                values = column_i
            values = [str(i) for i in values]
            values = " [" + (', '.join(values)) + ']'
            print(str(i+1) + ': ' + str(study_filtered[i]) + values)
            
    elif var_type == 'n':
        study_filtered = choose_data_type(study_q_results, 'number')
        for i in range(len(study_filtered)):
            column_i = study_q_results[study_filtered[i]]
            min_max_list = [min(study_q_results[study_filtered[i]]), max(study_q_results[study_filtered[i]])]
            values = min_max_list
            print(str(i+1) + ': ' + str(study_filtered[i]) + '; (min, max): (' + str(values[0]) + ', ' + str(values[1]) + ')') 
        
    print('')
    
    
    ## Put an if statememnt if it is empty
    
    ## Add a reset statement
    filter_more = True
    study_filtered_og = study_filtered
    while filter_more:
        filter_more = input('Do you want to filter the columns more? if yes, enter the keyword, otherwise enter NA, to go back enter Reset')
        
        if filter_more == 'NA':
            filter_more = False
            
        elif filter_more == 'Reset':
            study_filtered = study_filtered_og
            if var_type == 'c':
                print('')
                for i in range(len(study_filtered)):
                    column_i = study_q_results[study_filtered[i]].unique()
                    if len(column_i) > 5:
                        values = column_i[0:4]
                    else:
                        values = column_i
                    values = [str(i) for i in values]
                    values = " [" + (', '.join(values)) + ']'
                    print(str(i+1) + ': ' + str(study_filtered[i]) + values)
            
            elif var_type == 'n':
                for i in range(len(study_filtered)):
                    column_i = study_q_results[study_filtered[i]]
                    min_max_list = [min(study_q_results[study_filtered[i]]), max(study_q_results[study_filtered[i]])]
                    values = min_max_list
                    print(str(i+1) + ': ' + str(study_filtered[i]) + '; (min, max): (' + str(values[0]) + ', ' + str(values[1]) + ')') 
            
        else:
            tf_filtered = pd.Index(study_filtered).str.contains(filter_more, case = False)
            study_filtered_1 = []
            for i in range(len(tf_filtered)):
                if tf_filtered[i] == True:
                    study_filtered_1.append(study_filtered[i])
                    
            if len(study_filtered_1) == 0:
                print('No Matching Columns')
            else:
                study_filtered = study_filtered_1
                
            if var_type == 'c':
                print('')
                for i in range(len(study_filtered)):
                    column_i = study_q_results[study_filtered[i]].unique()
                    if len(column_i) > 5:
                        values = column_i[0:4]
                    else:
                        values = column_i
                    values = [str(i) for i in values]
                    values = " [" + (', '.join(values)) + ']'
                    print(str(i+1) + ': ' + str(study_filtered[i]) + values)
            
            elif var_type == 'n':
                for i in range(len(study_filtered)):
                    column_i = study_q_results[study_filtered[i]]
                    min_max_list = [min(study_q_results[study_filtered[i]]), max(study_q_results[study_filtered[i]])]
                    values = min_max_list
                    print(str(i+1) + ': ' + str(study_filtered[i]) + '; (min, max): (' + str(values[0]) + ', ' + str(values[1]) + ')') 
                
    var_index_flag = False
    var_index = input('Select variable number you want to pull data from: ')
    while var_index_flag == False:
        if var_index.isnumeric() == False:
            print('Please enter a number')
            var_index = input('Select variable number you want to pull data from: ')
            
        elif var_index.isnumeric() == True:
            var_index = int(var_index)
            var_index_flag = True
            break
    
    output_df = study_q_results.loc[:, ['Patient ID', study_filtered[var_index - 1 ]]]
    var_name = input('Name this variable: ')
    output_df.rename(columns = {study_filtered[var_index - 1 ]: var_name}, inplace = True)
    flag = True
    while flag:
        new_column = input("Do you want to add another column, yes or no?: ")
        if new_column == 'yes':
            var_type = input('Do you want Categorical(c) or Numerical Data(n)?')
            if var_type == 'c':
                study_filtered = choose_data_type(study_q_results, 'object')
                print('')
                for i in range(len(study_filtered)):
                    column_i = study_q_results[study_filtered[i]].unique()
                    if len(column_i) > 5:
                        values = column_i[0:4]
                    else:
                        values = column_i
                    values = [str(i) for i in values]
                    values = " [" + (', '.join(values)) + ']'
                    print(str(i+1) + ': ' + str(study_filtered[i]) + values)
                        
            elif var_type == 'n':
                study_filtered = choose_data_type(study_q_results, 'number')
                print('')
                for i in range(len(study_filtered)):
                    column_i = study_q_results[study_filtered[i]]
                    min_max_list = [min(study_q_results[study_filtered[i]]), max(study_q_results[study_filtered[i]])]
                    values = min_max_list
                    print(str(i+1) + ': ' + str(study_filtered[i]) + '; (min, max): (' + str(values[0]) + ', ' + str(values[1]) + ')') 

#             print('')
#             for i in range(len(study_filtered)):
#                 print(str(i+1) + ': ' + str(study_filtered[i]))
                
            print('')
            filter_more = True
            study_filtered_og = study_filtered
            while filter_more:
                filter_more = input('Do you want to filter the columns more? if yes, enter the keyword, otherwise enter NA, to go back enter Reset')
                
                if filter_more == 'NA':
                    filter_more = False

                elif filter_more == 'Reset':
                    study_filtered = study_filtered_og
                    if var_type == 'c':
                        print('')
                        for i in range(len(study_filtered)):
                            column_i = study_q_results[study_filtered[i]].unique()
                            if len(column_i) > 5:
                                values = column_i[0:4]
                            else:
                                values = column_i
                            values = [str(i) for i in values]
                            values = " [" + (', '.join(values)) + ']'
                            print(str(i+1) + ': ' + str(study_filtered[i]) + values)

                    elif var_type == 'n':
                        for i in range(len(study_filtered)):
                            column_i = study_q_results[study_filtered[i]]
                            min_max_list = [min(study_q_results[study_filtered[i]]), max(study_q_results[study_filtered[i]])]
                            values = min_max_list
                            print(str(i+1) + ': ' + str(study_filtered[i]) + '; (min, max): (' + str(values[0]) + ', ' + str(values[1]) + ')') 

                else:
                    tf_filtered = pd.Index(study_filtered).str.contains(filter_more, case = False)
                    study_filtered_1 = []
                    for i in range(len(tf_filtered)):
                        if tf_filtered[i] == True:
                            study_filtered_1.append(study_filtered[i])
                    if len(study_filtered_1) == 0:
                        print('No Matching Columns')
                    else:
                        study_filtered = study_filtered_1

                    if var_type == 'c':
                        print('')
                        for i in range(len(study_filtered)):
                            column_i = study_q_results[study_filtered[i]].unique()
                            if len(column_i) > 5:
                                values = column_i[0:4]
                            else:
                                values = column_i
                            values = [str(i) for i in values]
                            values = " [" + (', '.join(values)) + ']'
                            print(str(i+1) + ': ' + str(study_filtered[i]) + values)

                    elif var_type == 'n':
                        for i in range(len(study_filtered)):
                            column_i = study_q_results[study_filtered[i]]
                            min_max_list = [min(study_q_results[study_filtered[i]]), max(study_q_results[study_filtered[i]])]
                            values = min_max_list
                            print(str(i+1) + ': ' + str(study_filtered[i]) + '; (min, max): (' + str(values[0]) + ', ' + str(values[1]) + ')') 
                

            var_index = str(input('Select variable number you want to pull data from or type NA: '))
            
            #Fix this part in case there is a typo
            if var_index == 'NA':
                return output_df
            
            else:
                var_index = int(var_index)
                col_name = study_filtered[var_index - 1]
                output_df[str(col_name)] =  study_q_results[study_filtered[var_index - 1 ]]
                var_name = input('Name this variable: ')
                output_df.rename(columns = {study_filtered[var_index - 1 ]: var_name}, inplace = True)
            
        else:
            flag = False
    
    
    return output_df
    